In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from wordcloud import WordCloud
import missingno


%matplotlib inline
import random
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,CategoricalNB
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer


In [ ]:
from sklearn import preprocessing
from sklearn.manifold import TSNE
import seaborn as sns
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression

In [ ]:
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from time import time
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection

In [ ]:
train = pd.read_csv(r"D:\Projects\spam\train.csv")
test= pd.read_csv(r"D:\Projects\spam\test.csv")

In [ ]:
#checking for null values
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
#Remove redundant samples
train=train.drop_duplicates(subset=['text', 'target'], keep='first')
train.shape


In [ ]:
# drop location and keyword column
train = train.drop(['location','keyword'],axis=1)
test = test.drop(['location','keyword'],axis=1)

In [ ]:
#Identify common words: Its the frequently used words as well as it could be potential data specific stop words.
import pandas as pd

freq = pd.Series(' '.join(train['text']).split()).value_counts()[:20]
freq

In [ ]:
#Identify uncommon words: Uncommon words in the train dataset
freq1 =  pd.Series(' '.join(train ['text']).split()).value_counts()[-20:]
freq1

In [ ]:
#Disaster tweet

disaster_tweets = train[train['target'] ==1 ]['text']
for i in range(1,10):
    print(disaster_tweets[i])

In [ ]:
# non-disaster tweets
non_disaster_tweets = train[train['target'] !=1 ]['text']
non_disaster_tweets

In [ ]:
#wordcloud

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[20, 5])
wordcloud1 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(disaster_tweets))
ax1.imshow(wordcloud1)
ax1.axis('off')
ax1.set_title('Disaster Tweets',fontsize=20);


wordcloud2 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(non_disaster_tweets))
ax2.imshow(wordcloud2)
ax2.axis('off')
ax2.set_title('Non Disaster Tweets',fontsize=20);

In [ ]:
import nltk
nltk.download('stopwords')


In [ ]:
import string

def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
train['text'] = train['text'].apply(lambda x: clean_text(x))
test['text'] = test['text'].apply(lambda x: clean_text(x))

In [ ]:
train['text'].head()

In [ ]:
tweets = train["text"]

In [ ]:
#Tokenize


tokenizer=nltk.tokenize.RegexpTokenizer(r'\w+')
train['text'] = train['text'].apply(lambda x:tokenizer.tokenize(x))
test['text'] = test['text'].apply(lambda x:tokenizer.tokenize(x))
train['text'].head()

In [ ]:
#Removing stopwords


def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words 
train['text'] = train['text'].apply(lambda x : remove_stopwords(x))
test['text'] = test['text'].apply(lambda x : remove_stopwords(x))
test.head()

In [ ]:
disaster_tweets = train[train['target'] ==1 ]['text']
for i in range(1,10):
    print(disaster_tweets[i])

In [ ]:
# non-disaster tweets
non_disaster_tweets = train[train['target'] !=1 ]['text']
non_disaster_tweets.head()

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [ ]:
# lemmatization
lem = WordNetLemmatizer()
def lem_word(x):
    return [lem.lemmatize(w) for w in x]

In [ ]:
train['text'] = train['text'].apply(lem_word)
test['text'] = test['text'].apply(lem_word)

In [ ]:
def combine_text(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text

train['text'] = train['text'].apply(lambda x : combine_text(x))
test['text'] = test['text'].apply(lambda x : combine_text(x))
train['text']
train.head()

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
stem = PorterStemmer()

nltk.download()
#CountVectorizer
word_vectorizer.fit_transform(text_issue_list.split('\n'))
count_vectorizer = CountVectorizer()
train_vector = count_vectorizer.fit_transform(train['text'])
test_vector = count_vectorizer.transform(test['text'])
print(train_vector[0].todense())

In [ ]:
#TF IDF

tfidf = TfidfVectorizer(min_df = 2,max_df = 0.5,ngram_range = (1,2))
train_tfidf = tfidf.fit_transform(train['text'])
test_tfidf = tfidf.transform(test['text'])

In [ ]:
test_tfidf

In [ ]:
mnb = MultinomialNB(alpha = 2.0)
scores_vector = model_selection.cross_val_score(mnb,train_vector,train['target'],cv = 10,scoring = 'f1')
print("score:",scores_vector)
scores_tfidf = model_selection.cross_val_score(mnb,train_tfidf,train['target'],cv = 10,scoring = 'f1')
print("score of tfidf:",scores_tfidf)

In [ ]:
mnb.get_params()

In [ ]:
lg = LogisticRegression(C = 1.0)
scores_vector = model_selection.cross_val_score(lg, train_vector, train["target"], cv = 5, scoring = "f1")
print("score:",scores_vector)
scores_tfidf = model_selection.cross_val_score(lg, train_tfidf, train["target"], cv = 5, scoring = "f1")
print("score of tfidf:",scores_tfidf)

In [ ]:
lg.get_params()

In [ ]:
mnb.fit(train_tfidf, train["target"])
y_pred = mnb.predict(test_tfidf)


In [ ]:
y_pred


In [ ]:
tweet = input(Enter a tweet)
prediction = mnb.predict(tweet)